# Hyperelasticity
Author: Jørgen S. Dokken and Garth N. Wells

This section shows how to solve the hyperelasticity problem for deformation of a beam.

We will also show how to create a constant boundary condition for a vector function space.

We start by importing DOLFINx and some additional dependencies.
Then, we create a slender cantilever consisting of hexahedral elements and create the function space `V` for our unknown.

In [1]:
from dolfinx import log, default_scalar_type
from dolfinx.fem.petsc import NonlinearProblem
from dolfinx.nls.petsc import NewtonSolver
import pyvista
import numpy as np
import ufl

from mpi4py import MPI
from dolfinx import fem, mesh, plot
L = 20.0
domain = mesh.create_box(MPI.COMM_WORLD, [[0.0, 0.0, 0.0], [L, 1, 1]], [20, 5, 5], mesh.CellType.hexahedron)
V = fem.functionspace(domain, ("Lagrange", 2, (domain.geometry.dim, )))

We create two python functions for determining the facets to apply boundary conditions to

In [2]:
def left(x):
    return np.isclose(x[0], 0)


def right(x):
    return np.isclose(x[0], L)


fdim = domain.topology.dim - 1
left_facets = mesh.locate_entities_boundary(domain, fdim, left)
right_facets = mesh.locate_entities_boundary(domain, fdim, right)

Next, we create a  marker based on these two functions

In [3]:
# Concatenate and sort the arrays based on facet indices. Left facets marked with 1, right facets with two
marked_facets = np.hstack([left_facets, right_facets])
marked_values = np.hstack([np.full_like(left_facets, 1), np.full_like(right_facets, 2)])
sorted_facets = np.argsort(marked_facets)
facet_tag = mesh.meshtags(domain, fdim, marked_facets[sorted_facets], marked_values[sorted_facets])

We then create a function for supplying the boundary condition on the left side, which is fixed.

In [4]:
u_bc = np.array((0,) * domain.geometry.dim, dtype=default_scalar_type)

To apply the boundary condition, we identity the dofs located on the facets marked by the `MeshTag`.

In [5]:
left_dofs = fem.locate_dofs_topological(V, facet_tag.dim, facet_tag.find(1))
bcs = [fem.dirichletbc(u_bc, left_dofs, V)]

Next, we define the body force on the reference configuration (`B`), and nominal (first Piola-Kirchhoff) traction (`T`).

In [6]:
B = fem.Constant(domain, default_scalar_type((0, 0, 0)))
T = fem.Constant(domain, default_scalar_type((0, 0, 0)))

Define the test and solution functions on the space $V$

In [7]:
v = ufl.TestFunction(V)
u = fem.Function(V)

Define kinematic quantities used in the problem

In [8]:
# Spatial dimension
d = len(u)

# Identity tensor
I = ufl.variable(ufl.Identity(d))

# Deformation gradient
F = ufl.variable(I + ufl.grad(u))

# Right Cauchy-Green tensor
C = ufl.variable(F.T * F)

# Invariants of deformation tensors
Ic = ufl.variable(ufl.tr(C))
J = ufl.variable(ufl.det(F))

Define the elasticity model via a stored strain energy density function $\psi$, and create the expression for the first Piola-Kirchhoff stress:

In [9]:
# Elasticity parameters
E = default_scalar_type(1.0e4)
nu = default_scalar_type(0.3)
mu = fem.Constant(domain, E / (2 * (1 + nu)))
lmbda = fem.Constant(domain, E * nu / ((1 + nu) * (1 - 2 * nu)))
# Stored strain energy density (compressible neo-Hookean model)
psi = (mu / 2) * (Ic - 3) - mu * ufl.ln(J) + (lmbda / 2) * (ufl.ln(J))**2
# Stress
# Hyper-elasticity
P = ufl.diff(psi, F)

```{admonition} Comparison to linear elasticity
To illustrate the difference between linear and hyperelasticity, the following lines can be uncommented to solve the linear elasticity problem.
```

In [10]:
# P = 2.0 * mu * ufl.sym(ufl.grad(u)) + lmbda * ufl.tr(ufl.sym(ufl.grad(u))) * I

Define the variational form with traction integral over all facets with value 2. We set the quadrature degree for the integrals to 4.

In [11]:
metadata = {"quadrature_degree": 4}
ds = ufl.Measure('ds', domain=domain, subdomain_data=facet_tag, metadata=metadata)
dx = ufl.Measure("dx", domain=domain, metadata=metadata)
# Define form F (we want to find u such that F(u) = 0)
F = ufl.inner(ufl.grad(v), P) * dx - ufl.inner(v, B) * dx - ufl.inner(v, T) * ds(2)

As the varitional form is non-linear and written on residual form, we use the non-linear problem class from DOLFINx to set up required structures to use a Newton solver.

In [12]:
problem = NonlinearProblem(F, u, bcs)

and then create and customize the Newton solver

In [13]:
solver = NewtonSolver(domain.comm, problem)

# Set Newton solver options
solver.atol = 1e-8
solver.rtol = 1e-8
solver.convergence_criterion = "incremental"


We create a function to plot the solution at each time step.

In [17]:
from pathlib import Path
import os
current_directory = os.getcwd()
results_folder = Path(current_directory + '/fenicsx/hyperelasticity/results/')
results_folder.mkdir(exist_ok=True, parents=True)

pyvista.start_xvfb()
plotter = pyvista.Plotter()
plotter.open_gif(str(results_folder) + "/displacement.gif", fps=3)

topology, cells, geometry = plot.vtk_mesh(u.function_space)
function_grid = pyvista.UnstructuredGrid(topology, cells, geometry)

values = np.zeros((geometry.shape[0], 3))
values[:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
function_grid["u"] = values
function_grid.set_active_vectors("u")

# Warp mesh by deformation
warped = function_grid.warp_by_vector("u", factor=1)
warped.set_active_vectors("u")

# Add mesh to plotter and visualize
actor = plotter.add_mesh(warped, show_edges=True, lighting=False, clim=[0, 10])

# Compute magnitude of displacement to visualize in GIF
Vs = fem.functionspace(domain, ("Lagrange", 2))
magnitude = fem.Function(Vs)
us = fem.Expression(ufl.sqrt(sum([u[i]**2 for i in range(len(u))])), Vs.element.interpolation_points())
magnitude.interpolate(us)
warped["mag"] = magnitude.x.array

Finally, we solve the problem over several time steps, updating the z-component of the traction

In [18]:
log.set_log_level(log.LogLevel.INFO)
tval0 = -1.5
for n in range(1, 10):
    T.value[2] = n * tval0
    num_its, converged = solver.solve(u)
    assert (converged)
    u.x.scatter_forward()
    print(f"Time step {n}, Number of iterations {num_its}, Load {T.value}")
    function_grid["u"][:, :len(u)] = u.x.array.reshape(geometry.shape[0], len(u))
    magnitude.interpolate(us)
    warped.set_active_scalars("mag")
    warped_n = function_grid.warp_by_vector(factor=1)
    warped.points[:, :] = warped_n.points
    warped.point_data["mag"][:] = magnitude.x.array
    plotter.update_scalar_bar_range([0, 10])
    plotter.write_frame()
plotter.close()

2024-09-17 11:54:25.877 ( 352.514s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:26.310 ( 352.947s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:26.552 ( 353.189s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 22.2455 (tol = 1e-08) r (rel) = 0.134278(tol = 1e-08)
2024-09-17 11:54:26.672 ( 353.309s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:26.913 ( 353.550s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 2.43261 (tol = 1e-08) r (rel) = 0.0146837(tol = 1e-08)
2024-09-17 11:54:27.048 ( 353.685s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:27.282 ( 353.919s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 

Time step 1, Number of iterations 8, Load [ 0.   0.  -1.5]


2024-09-17 11:54:28.962 ( 355.599s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:29.328 ( 355.966s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:29.575 ( 356.213s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 17.3254 (tol = 1e-08) r (rel) = 0.117842(tol = 1e-08)
2024-09-17 11:54:29.699 ( 356.337s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:29.937 ( 356.575s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 5.14882 (tol = 1e-08) r (rel) = 0.0350207(tol = 1e-08)
2024-09-17 11:54:30.061 ( 356.699s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:30.335 ( 356.972s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 

Time step 2, Number of iterations 9, Load [ 0.  0. -3.]


2024-09-17 11:54:32.951 ( 359.588s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:33.314 ( 359.952s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 10.0011 (tol = 1e-08) r (rel) = 0.0887471(tol = 1e-08)
2024-09-17 11:54:33.454 ( 360.091s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:33.703 ( 360.340s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 5.33026 (tol = 1e-08) r (rel) = 0.0472992(tol = 1e-08)
2024-09-17 11:54:33.832 ( 360.470s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:34.097 ( 360.734s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: r (abs) = 11.9901 (tol = 1e-08) r (rel) = 0.106397(tol = 1e-08)
2024-09-17 11:54:34.225 ( 360.863s) [main            ]              

...

Time step 7, Number of iterations 6, Load [  0.    0.  -10.5]


2024-09-17 11:54:49.426 ( 376.063s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 6: r (abs) = 1.7886e-08 (tol = 1e-08) r (rel) = 4.34697e-10(tol = 1e-08)
2024-09-17 11:54:49.426 ( 376.063s) [main            ]       NewtonSolver.cpp:255   INFO| Newton solver finished in 6 iterations and 6 linear solver iterations.
2024-09-17 11:54:49.610 ( 376.247s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:50.031 ( 376.669s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:50.310 ( 376.947s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 1.06336 (tol = 1e-08) r (rel) = 0.031085(tol = 1e-08)
2024-09-17 11:54:50.447 ( 377.085s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:50.709 ( 377.346s) [main            ]       NewtonSolver.cpp

Time step 8, Number of iterations 6, Load [  0.   0. -12.]


2024-09-17 11:54:52.611 ( 379.248s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:53.018 ( 379.655s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 0.898789 (tol = 1e-08) r (rel) = 0.0309666(tol = 1e-08)
2024-09-17 11:54:53.177 ( 379.814s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:53.513 ( 380.150s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 1.38354 (tol = 1e-08) r (rel) = 0.0476679(tol = 1e-08)
2024-09-17 11:54:53.655 ( 380.292s) [main            ]              petsc.cpp:698   INFO| PETSc Krylov solver starting to solve system.
2024-09-17 11:54:53.937 ( 380.574s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: r (abs) = 0.00185096 (tol = 1e-08) r (rel) = 6.37724e-05(tol = 1e-08)
2024-09-17 11:54:54.077 ( 380.714s) [main            ]       

Time step 9, Number of iterations 6, Load [  0.    0.  -13.5]


![Deformation](results/displacement.gif)
